In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import os
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
data = pd.read_excel('data_orig.xlsx', index_col=0)

In [3]:
data

,GSM658075,GSM658076,GSM658077,GSM658078,GSM658079,GSM658080,GSM658081,GSM658082,GSM658083,GSM658084,GSM658085,GSM658086,GSM658087,GSM658088,GSM658089,GSM658090,GSM658091,GSM658092
ID_REF,,,,,,,,,,,,,,,,,,
A_51_P100034,12.630,11.899,12.393,12.539,12.368,12.361,12.332,12.148,12.032,11.955,12.222,12.202,12.107,12.529,11.838,12.054,12.223,12.343
A_51_P100063,5.828,6.598,5.984,5.986,6.089,6.163,6.384,6.410,5.984,6.158,6.031,5.833,6.441,7.814,6.107,7.017,5.853,6.656
A_51_P100099,7.638,7.635,7.216,7.767,7.516,7.378,7.171,7.558,7.571,7.607,7.523,7.628,7.461,7.321,7.792,7.577,7.412,7.357
A_51_P100155,10.947,10.582,10.406,11.054,10.770,10.402,10.610,10.512,10.492,10.485,10.839,10.535,10.674,10.333,10.622,10.302,10.339,10.572
A_51_P100174,6.632,6.089,6.012,6.190,6.260,6.131,5.993,6.274,6.002,6.011,6.124,6.062,6.375,6.087,6.152,6.202,6.209,6.172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A_52_P997209,7.734,8.105,7.841,7.631,7.893,8.232,7.570,8.146,7.550,7.820,7.648,7.675,7.763,8.154,7.703,7.878,8.058,7.989
A_52_P997449,10.070,9.784,9.009,10.071,9.762,9.535,9.876,9.999,10.084,10.064,10.219,10.133,9.428,9.136,9.890,10.302,10.493,9.447
A_52_P99807,8.176,7.628,7.993,7.717,6.355,6.668,6.699,7.256,6.803,8.208,8.409,7.008,6.384,8.434,6.441,6.719,6.401,7.256


In [4]:
data_full = pd.read_table('data/GSM658075_C1.txt', skiprows=9)
data_full = data_full.iloc[:, :60]
data_full.iloc[:11, :]

,FEATURES,FeatureNum,Row,Col,accessions,chr_coord,SubTypeMask,SubTypeName,ProbeUID,ControlType,...,SpotExtentX,SpotExtentY,gNetSignal,gMultDetrendSignal,gProcessedBackground,gProcessedBkngError,IsUsedBGAdjust,gInterpolatedNegCtrlSub,gIsInNegCtrlRange,gIsUsedInMD
0,DATA,1,1,1,NaN,,260,BrightCorner,0,1,...,54.9904,54.9904,1706.2800,1.02250,33.0636,8.06569,0,1668.770000,0,0
1,DATA,2,1,2,NaN,,66,Structural,1,1,...,42.5954,42.5954,37.5397,1.02849,33.4136,8.16225,0,0.045400,0,0
2,DATA,3,1,3,NaN,,66,Structural,1,1,...,65.0000,65.0000,37.4516,1.03475,32.5570,7.59847,0,-0.023116,0,0
3,DATA,4,1,4,NaN,,66,Structural,1,1,...,47.8731,47.8731,36.1639,1.04018,32.1337,7.35055,0,-1.291920,0,0
4,DATA,5,1,5,NaN,,66,Structural,1,1,...,65.0000,65.0000,35.3016,1.04529,31.6800,7.56199,0,-2.135510,0,0
5,DATA,6,1,6,NaN,,66,Structural,1,1,...,46.8651,46.8651,34.9063,1.05026,31.9504,7.63066,0,-2.512130,0,0
6,DATA,7,1,7,NaN,,66,Structural,1,1,...,57.5363,57.5363,36.9016,1.05440,32.2364,7.73532,0,-0.498250,0,0
7,DATA,8,1,8,NaN,,66,Structural,1,1,...,65.0000,65.0000,37.1846,1.05834,31.9624,7.36118,0,-0.197140,0,0
8,DATA,9,1,9,NaN,,66,Structural,1,1,...,57.5363,57.5363,39.4923,1.06210,31.9750,7.36294,0,2.128690,0,0
9,DATA,10,1,10,NaN,,66,Structural,1,1,...,65.0000,65.0000,35.1475,1.06540,32.4278,7.78089,0,-2.198350,0,0


In [5]:
data_full.shape

(45018, 60)

In [6]:
# looks like same :)
d1 = pd.read_table('data/GSM658075_C1.txt', skiprows=9)

for filename in os.listdir('data'):
    d2 = pd.read_table(os.path.join('data', filename), skiprows=9)
    print((d1 == d2)[['ProbeName', 'GeneName']].sum() - [d1.shape[0]]*2)


ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64
ProbeName    0
GeneName     0
dtype: int64


In [7]:
# validate if all probes in C1 table
k = 0
for el in data.index:
    if not el in data_full['ProbeName'].values:
        k += 1
print(k)

0


In [8]:
indexes = []
for i, el in enumerate(data_full['ProbeName'].values):
    if el in data.index:
        indexes.append(i)

In [9]:
clear_data_full = data_full.loc[indexes]

In [10]:
len(indexes), np.unique(clear_data_full['ProbeName']).shape, data.index.shape

(24026, (22514,), (22514,))

In [11]:
clear_data_full['ProbeName'].describe()

count            24026
unique           22514
top       A_51_P109469
freq                10
Name: ProbeName, dtype: object

In [12]:
tmp_df = clear_data_full[clear_data_full['ProbeName'] == 'A_52_P148861']
tmp_df

,FEATURES,FeatureNum,Row,Col,accessions,chr_coord,SubTypeMask,SubTypeName,ProbeUID,ControlType,...,SpotExtentX,SpotExtentY,gNetSignal,gMultDetrendSignal,gProcessedBackground,gProcessedBkngError,IsUsedBGAdjust,gInterpolatedNegCtrlSub,gIsInNegCtrlRange,gIsUsedInMD
2581,DATA,2594,31,44,ref|NM_009743|gb|AK146461|gb|AK150028|ens|ENSM...,chr2:152472395-152472336,0,NaN,2286,0,...,49.1849,49.1849,194.429,1.120120,32.7619,6.86387,0,157.2420,0,1
3033,DATA,3047,36,72,ref|NM_009743|gb|AK146461|gb|AK150028|ens|ENSM...,chr2:152472395-152472336,0,NaN,2286,0,...,57.5363,57.5363,195.773,1.118740,35.2800,8.28915,0,158.7130,0,1
3551,DATA,3566,42,81,ref|NM_009743|gb|AK146461|gb|AK150028|ens|ENSM...,chr2:152472395-152472336,0,NaN,2286,0,...,46.8651,46.8651,213.554,1.100960,35.9722,7.51945,0,176.4460,0,1
6664,DATA,6698,79,68,ref|NM_009743|gb|AK146461|gb|AK150028|ens|ENSM...,chr2:152472395-152472336,0,NaN,2286,0,...,48.8603,48.8603,214.953,1.170390,34.3704,10.48860,0,177.4710,0,1
7787,DATA,7824,93,4,ref|NM_009743|gb|AK146461|gb|AK150028|ens|ENSM...,chr2:152472395-152472336,0,NaN,2286,0,...,47.5395,47.5395,194.113,1.040610,33.6429,7.45959,0,156.0860,0,1
12943,DATA,13009,154,4,ref|NM_009743|gb|AK146461|gb|AK150028|ens|ENSM...,chr2:152472395-152472336,0,NaN,2286,0,...,45.4864,45.4864,165.391,0.988923,33.6154,7.57404,0,127.3360,0,1
18844,DATA,18934,223,64,ref|NM_009743|gb|AK146461|gb|AK150028|ens|ENSM...,chr2:152472395-152472336,0,NaN,2286,0,...,47.8731,47.8731,216.065,1.199090,35.4286,7.96683,0,178.2500,0,1
27844,DATA,27970,330,5,ref|NM_009743|gb|AK146461|gb|AK150028|ens|ENSM...,chr2:152472395-152472336,0,NaN,2286,0,...,46.8651,46.8651,148.394,0.782246,38.0769,10.16240,0,111.7780,0,1
44570,DATA,44772,527,62,ref|NM_009743|gb|AK146461|gb|AK150028|ens|ENSM...,chr2:152472395-152472336,0,NaN,2286,0,...,45.8350,45.8350,148.277,0.684011,34.3061,8.91442,0,114.8630,0,1
44977,DATA,45179,532,44,ref|NM_009743|gb|AK146461|gb|AK150028|ens|ENSM...,chr2:152472395-152472336,0,NaN,2286,0,...,48.5334,48.5334,126.226,0.650284,32.0244,7.62720,0,93.3591,0,1


In [13]:
columns_names = tmp_df.columns
len(columns_names), tmp_df.shape

(60, (10, 60))

In [14]:
for i, col in enumerate(tmp_df):
#     print(tmp_df[col].astype(str))
    uni = np.unique(tmp_df[col].astype(str))
    if len(uni) > 1:
        print(uni.shape, columns_names[i] , uni, end='\n_______\n')

(10,) FeatureNum ['13009' '18934' '2594' '27970' '3047' '3566' '44772' '45179' '6698'
 '7824']
_______
(10,) Row ['154' '223' '31' '330' '36' '42' '527' '532' '79' '93']
_______
(8,) Col ['4' '44' '5' '62' '64' '68' '72' '81']
_______
(10,) PositionX ['2001.34' '2013.17' '2034.91' '3018.19' '3020.75' '3465.21' '3522.51'
 '3626.99' '3742.33' '3970.33']
_______
(10,) PositionY ['1202.34' '1710.93' '2492.1' '302.392' '340.5' '385.17400000000004'
 '3938.38' '3974.5' '655.6160000000001' '755.38']
_______
(10,) gProcessedSignal ['128.2583' '139.3703' '140.6332' '141.2245' '142.2376' '147.8046'
 '149.7119' '150.6274' '159.0971' '166.3322']
_______
(10,) gProcessedSigError ['12.93133' '14.01281' '14.13861' '14.34322' '14.37561' '14.84285'
 '15.05292' '15.127' '15.97846' '16.803079999999998']
_______
(4,) gNumPixOLHi ['0' '1' '2' '4']
_______
(5,) gNumPix ['62' '63' '64' '65' '66']
_______
(10,) gMeanSignal ['126.2258' '148.2769' '148.3939' '165.3906' '194.1129' '194.4286'
 '195.7727' '213.5538

In [15]:
data_and_gene = clear_data_full[['ProbeName', 'GeneName']].merge(data, left_on=['ProbeName'], right_index=True)
data_and_gene = data_and_gene.drop_duplicates().set_index('ProbeName')
data_and_gene

,GeneName,GSM658075,GSM658076,GSM658077,GSM658078,GSM658079,GSM658080,GSM658081,GSM658082,GSM658083,GSM658084,GSM658085,GSM658086,GSM658087,GSM658088,GSM658089,GSM658090,GSM658091,GSM658092
ProbeName,,,,,,,,,,,,,,,,,,,
A_52_P580582,Nppa,5.882,5.609,5.760,5.764,5.840,5.868,5.697,5.659,5.847,5.789,5.861,5.939,5.762,5.719,5.830,5.800,6.061,5.789
A_52_P819156,AK046412,5.986,5.917,5.697,5.619,5.914,5.800,5.782,6.765,5.702,5.954,5.937,6.062,5.608,6.184,5.896,5.929,5.653,5.619
A_51_P331831,Hvcn1,6.584,6.031,6.553,5.738,6.470,6.073,6.137,6.038,5.866,5.869,6.535,5.845,6.354,5.913,6.420,5.967,6.401,6.260
A_52_P684402,Ptdss2,7.875,7.446,7.882,7.700,7.512,7.626,7.362,7.179,7.344,7.698,7.708,7.464,7.687,7.230,7.831,7.205,7.659,7.221
A_51_P280918,Itfg1,8.311,7.731,7.885,7.858,7.766,7.863,7.587,8.513,7.940,8.153,8.081,7.780,8.051,7.826,8.240,7.761,7.975,7.243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A_52_P321627,LOC674226,12.062,13.422,13.074,12.486,12.700,13.253,13.229,13.243,13.275,13.113,12.925,13.031,13.198,13.078,12.840,13.386,13.393,13.220
A_51_P521176,Rassf7,7.902,10.266,10.375,10.354,10.077,10.260,10.859,10.614,10.407,10.502,10.177,10.483,10.225,10.021,10.272,10.612,10.156,10.382
A_52_P189897,Gtf2a2,5.882,10.263,10.072,10.226,10.028,10.306,9.863,10.652,10.326,10.378,9.818,9.983,10.244,10.490,10.353,10.174,10.192,10.177


In [16]:
# A
# number of genes 
gene_names = np.unique(data_and_gene['GeneName'].astype(str))
gene_names.shape

(15344,)

In [17]:
# B
tmpd = data_and_gene.groupby(['GeneName']).count()['GSM658075']
print(tmpd.sort_values(ascending=False))
print(tmpd[tmpd > 1].shape)

GeneName
Usp15            7
Ablim1           6
Pde4dip          6
Rb1cc1           6
Brwd1            6
                ..
Mid1             1
Micall2          1
Micall1          1
Mical3           1
0610005C13Rik    1
Name: GSM658075, Length: 15344, dtype: int64
(5164,)


In [18]:
mean_dag = data_and_gene.groupby(['GeneName']).mean()
mean_dag

,GSM658075,GSM658076,GSM658077,GSM658078,GSM658079,GSM658080,GSM658081,GSM658082,GSM658083,GSM658084,GSM658085,GSM658086,GSM658087,GSM658088,GSM658089,GSM658090,GSM658091,GSM658092
GeneName,,,,,,,,,,,,,,,,,,
0610005C13Rik,12.3860,12.3740,12.3850,12.7220,12.5500,12.471,11.6480,12.4190,12.1840,12.5640,12.0920,12.1990,12.4510,12.5540,12.657,12.2810,12.3450,12.7270
0610006I08Rik,11.3490,11.8860,11.7660,11.5750,11.6320,11.753,11.4950,11.7340,11.7830,11.5930,11.2380,11.7150,11.6920,11.6590,11.759,11.8670,11.7420,11.7140
0610007C21Rik,12.5980,12.4800,12.6690,12.7640,12.7440,12.315,12.2160,12.6300,12.5390,12.5120,12.3280,12.4330,12.7620,12.7340,12.529,12.6010,12.4670,12.6810
0610007L01Rik,11.1325,10.7675,10.8120,10.9670,10.8690,10.881,11.0265,11.0810,10.7770,11.2030,11.3310,10.9930,10.8200,11.3445,10.598,11.0810,10.6630,10.7625
0610007N19Rik,7.9740,7.7010,8.2270,7.9570,7.8680,7.784,8.3460,8.1000,7.7990,8.0240,7.9470,7.9140,7.8880,7.5330,8.224,7.9840,8.2030,7.8330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zzef1,6.1580,6.7040,6.4380,6.3310,6.2860,6.122,6.3550,6.2010,6.3570,6.2020,6.3660,5.9620,6.2680,6.0650,6.468,6.3950,6.1840,6.1400
Zzz3,8.2340,8.2820,8.2010,8.4260,8.0780,8.185,8.1080,8.5080,8.3800,8.3440,8.1470,8.3290,8.2750,8.2070,8.499,8.4930,8.0910,7.7560
pPtp4a3,7.0105,6.4455,6.7805,6.6565,6.9860,6.906,6.8070,6.6830,6.8450,6.7180,6.7085,6.8785,7.0040,6.6345,6.830,6.7395,6.9095,6.8690


In [40]:
# C
tmp_dict = {}
for i, name in enumerate(('control', 'low', 'high')):
    tmp_dict[name] = mean_dag.iloc[:,i:i+6].mean(axis=1)
pd.DataFrame(tmp_dict)

,control,low,high
GeneName,,,
0610005C13Rik,12.481333,12.358333,12.365833
0610006I08Rik,11.660167,11.684500,11.659167
0610007C21Rik,12.595000,12.531333,12.556333
0610007L01Rik,10.904833,10.887167,10.939417
0610007N19Rik,7.918500,7.980500,8.047000
...,...,...,...
Zzef1,6.339833,6.372667,6.288833
Zzz3,8.234333,8.213333,8.251000
pPtp4a3,6.797500,6.763583,6.803167


In [19]:
transpoded_mean_dag = mean_dag.T
transpoded_mean_dag

GeneName,0610005C13Rik,0610006I08Rik,0610007C21Rik,0610007L01Rik,0610007N19Rik,0610007P06Rik,0610007P08Rik,0610007P14Rik,0610007P22Rik,0610008C08Rik,...,Zxdc,Zyg11a,Zyg11b,Zyg11bl,Zyx,Zzef1,Zzz3,pPtp4a3,rp9,zfp507
GSM658075,12.386,11.349,12.598,11.1325,7.974,10.177,5.828,13.538,10.220,11.801,...,7.2665,5.940,7.2850,7.2960,11.940,6.158,8.234,7.0105,12.178,6.6935
GSM658076,12.374,11.886,12.480,10.7675,7.701,10.125,6.031,13.635,10.472,11.310,...,7.2395,6.261,7.3945,7.6505,11.482,6.704,8.282,6.4455,12.483,6.8320
GSM658077,12.385,11.766,12.669,10.8120,8.227,10.005,5.772,13.741,10.437,11.738,...,7.1905,5.822,7.2955,7.5585,11.833,6.438,8.201,6.7805,12.423,6.7295
GSM658078,12.722,11.575,12.764,10.9670,7.957,10.570,5.864,13.737,10.417,11.807,...,7.2510,5.864,7.5175,7.3845,11.747,6.331,8.426,6.6565,12.136,6.8325
GSM658079,12.550,11.632,12.744,10.8690,7.868,10.140,5.914,13.671,10.256,11.594,...,7.1930,6.033,7.4705,7.3345,11.796,6.286,8.078,6.9860,12.129,6.5255
GSM658080,12.471,11.753,12.315,10.8810,7.784,9.860,5.840,13.016,10.614,11.475,...,7.2595,5.828,7.6460,7.5765,11.559,6.122,8.185,6.9060,12.623,6.8930
GSM658081,11.648,11.495,12.216,11.0265,8.346,10.164,5.870,13.872,10.088,11.561,...,7.1505,5.848,6.9670,7.4610,11.481,6.355,8.108,6.8070,12.158,6.7140
GSM658082,12.419,11.734,12.630,11.0810,8.100,10.089,5.907,13.409,10.568,11.443,...,7.3415,5.907,6.8710,7.5425,11.624,6.201,8.508,6.6830,12.552,6.8555
GSM658083,12.184,11.783,12.539,10.7770,7.799,10.333,5.613,13.811,10.406,11.589,...,7.1820,6.073,6.9615,7.3715,11.508,6.357,8.380,6.8450,12.426,6.6115
GSM658084,12.564,11.593,12.512,11.2030,8.024,10.202,6.063,13.603,10.426,11.637,...,7.2260,5.894,7.5275,7.6020,11.613,6.202,8.344,6.7180,12.346,6.8435


In [20]:
control = transpoded_mean_dag.iloc[0:6]
low = transpoded_mean_dag.iloc[6:12]
high = transpoded_mean_dag.iloc[12:18]

In [21]:
X_control_low = np.append(control, low, axis=0)
y_control_low = np.array([0]*control.shape[0] + [1]*low.shape[0])

X_control_high = np.append(control, high, axis=0)
y_control_high = np.array([0]*control.shape[0] + [1]*high.shape[0])

In [22]:
X_control_low.shape, y_control_low.shape

((12, 15344), (12,))

## Classifiers

In [23]:
clf_low = RandomForestClassifier(max_depth=2, n_estimators=100, n_jobs=-1)

cvs = cross_val_score(clf_low, X_control_low, y_control_low,
                      cv=6, 
                      n_jobs=-1,
                      scoring='roc_auc'
                     )
print('cross_val. mean: {} | {}'.format(cvs.mean(), cvs))

clf_low.fit(X_control_low, y_control_low)
sort_ind = np.argsort(clf_low.feature_importances_)[::-1]
gene_importances_low_df = \
    pd.DataFrame.from_records(list(zip(clf_low.feature_importances_[sort_ind], transpoded_mean_dag.columns[sort_ind])),
                              columns=['gene_importance', 'gene'], index=['gene'])

print('number of non-zero gene importance :', (gene_importances_low_df.gene_importance > 0).sum())
gene_importances_low_df

cross_val. mean: 1.0 | [1. 1. 1. 1. 1. 1.]
number of non-zero gene importance : 94


,gene_importance
gene,
Olfr374,0.02
AK041551,0.02
Rpia,0.02
Tmem29,0.02
Glcci1,0.02
...,...
Ntn2l,0.00
Ntn1,0.00
Nthl1,0.00


In [24]:
clf_high = RandomForestClassifier(max_depth=2, n_estimators=100, n_jobs=-1)

cvs = cross_val_score(clf_high, X_control_high, y_control_high, 
                      cv=6, 
                      n_jobs=-1,
                      scoring='roc_auc'
                     )
print('cross_val. mean: {} | {}'.format(cvs.mean(), cvs))

clf_high.fit(X_control_high, y_control_high)
sort_ind = np.argsort(clf_high.feature_importances_)[::-1]
gene_importances_high_df = \
    pd.DataFrame.from_records(list(zip(clf_high.feature_importances_[sort_ind], transpoded_mean_dag.columns[sort_ind])),
                              columns=['gene_importance', 'gene'], index=['gene'])

print('number of non-zero gene importance :', (gene_importances_high_df.gene_importance > 0).sum())
gene_importances_high_df

cross_val. mean: 0.8333333333333334 | [1. 1. 0. 1. 1. 1.]
number of non-zero gene importance : 103


,gene_importance
gene,
Arrdc3,0.02
2810046L04Rik,0.01
NAP121186-002,0.01
5430434G16Rik,0.01
Zfp30,0.01
...,...
Ntn1,0.00
Nthl1,0.00
Ntan1,0.00


In [33]:
# D
tmp_df = gene_importances_high_df.merge(gene_importances_low_df, left_index=True, right_index=True)
tmp_df[(tmp_df.gene_importance_x == tmp_df.gene_importance_y) & (tmp_df.gene_importance_x != 0)]

,gene_importance_x,gene_importance_y
gene,,
